In [1]:
import requests
import pandas as pd

import tqdm

import random
import time

from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv("data/casas_incompleto.csv")
print(df.shape)
df.head()

(16180, 4)


,Unnamed: 0,nome,valor_venda,url
0,0,Estância Quintas Da Alvorada Casa Térrea Const...,340000,https://www.wimoveis.com.br/propriedades/estan...
1,1,Casa | 3 Quartos | Piscina | Rua 8 Vicente Pires,700000,https://www.wimoveis.com.br/propriedades/casa-...
2,2,"Casa Com 3 Quartos 143,33 m² De Área Útil, Mod...",420000,https://www.wimoveis.com.br/propriedades/casa-...
3,3,Rua 4a - Excelente Casa - 03 Quartos - Fino Ac...,595000,https://www.wimoveis.com.br/propriedades/rua-4...
4,4,"Qe 28 Guará II, 109 m² - 02 Quartos",680000,https://www.wimoveis.com.br/propriedades/qe-28...


In [3]:
df = df.drop(['Unnamed: 0'],axis=1)
df['condominio'] = ''
df['IPTU'] = ''
df['lat'] = ''
df['long'] = ''
df['area_total'] = ''
df['area_util'] = ''
df['banheiros'] = ''
df['vagas'] = ''
df['quartos'] = ''
df['suites'] = ''
df['idade'] = ''
df['endereco'] = ''
df['bairro'] = ''
df['idade_anuncio'] = ''

In [4]:
SAVE_EVERY_N = 1000

In [7]:
for i, row in tqdm.tqdm_notebook(df.iterrows(), total=df.shape[0]):
    try:
        
        tqdm.tqdm.write(str(i + 1) +" Buscando item " + row.url)
        response = requests.get(row.url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Condominio e IPTU
        valores = soup.findAll("div",{"class":"block-expensas block-row"})
        if(len(valores)>0):
            for item in valores:
                #tqdm.tqdm.write(str(item))
                if("Condomínio" in str(item) ):
                    #print(item.find("span").text)
                    df.at[i,'condominio'] = item.find("span").text
                else:
                    df.at[i,'IPTU'] = item.find("span").text
                    
        # latlong
        script = soup.findAll("script")[2].string
        lat = script[script.find("mapLat") + 9:script.find("mapLat") + 29].replace(",","")
        df.at[i,'lat'] = lat
        
        
        long = script[script.find("mapLng") + 9:script.find("mapLng") + 29].replace(",","")
        df.at[i,'long'] = long
        
        # icones
        icon_total = soup.find("i",{"class":"icon-f icon-f-stotal"})
        if icon_total is not None:
            area_total = icon_total.findNext('b').text
            df.at[i,'area_total'] = area_total
        
        icon_util = soup.find("i",{"class":"icon-f icon-f-scubierta"})
        if icon_util is not None:
            area_util = icon_util.findNext('b').text
            df.at[i,'area_util'] = area_util
            
        icon_banheiro = soup.find("i",{"class":"icon-f icon-f-bano"})
        if icon_banheiro is not None:
            qtd_banheiros = icon_banheiro.findNext('b').text
            df.at[i,'banheiros'] = qtd_banheiros
            
        icon_vagas = soup.find("i",{"class":"icon-f icon-f-cochera"})
        if icon_vagas is not None:
            qtd_vagas = icon_vagas.findNext('b').text
            df.at[i,'vagas'] = qtd_vagas
        
        icon_quartos = soup.find("i",{"class":"icon-f icon-f-dormitorio"})
        if icon_quartos is not None:
            qtd_quartos = icon_quartos.findNext('b').text
            df.at[i,'quartos'] = qtd_quartos
        
        icon_suites = soup.find("i",{"class":"icon-f icon-f-toilete"})
        if icon_suites is not None:
            qtd_suites = icon_suites.findNext('b').text
            df.at[i,'suites'] = qtd_suites
        
        icon_idade = soup.find("i",{"class":"icon-f icon-f-antiguedad"})
        if icon_idade is not None:
            idade = icon_idade.findNext('b').text
            df.at[i,'idade'] = idade
        
        h2_endereco = soup.find("h2",{"class":"title-location"})
        if h2_endereco is not None:
            df.at[i,'endereco'] = h2_endereco.text
            df.at[i,'bairro'] = h2_endereco.findNext("span").text
        
        h5_idade = soup.find("h5",{"class":"section-date css-float-r"})
        if h5_idade is not None:
            df.at[i,'idade_anuncio'] = h5_idade.text.replace("\n Publicado há ","").replace(" dias\n","")
        
    except Exception as e:
        print("exceção ao buscar dados. Mensagem:", str(e))
        pass
    
    if (i % SAVE_EVERY_N) == 0:
        print("Salvando arquivo intermediario em data/casas_completo_intermediario.csv")
        df.to_csv("data/casas_completo_intermediario.csv")
    
    #dorme
    aleatorio = random.randint(1, 4)
    tqdm.tqdm.write("agora vou aguardar: " + str(aleatorio))
    time.sleep(aleatorio)

print(df.shape)
df.head(10)

1 Buscando item https://www.wimoveis.com.br/propriedades/estancia-quintas-da-alvorada-casa-terrea-construcao-2947102743.html
None
exceção ao buscar dados. Mensagem: 'NoneType' object has no attribute 'find'
Salvando arquivo intermediario em data/casas_completo_intermediario.csv
agora vou aguardar: 4



KeyboardInterrupt: 

In [ ]:
df.to_csv("data/apartamentos_completo.csv")